#  <span style="color:#3386cd; font-size:28px">Mapping DHS with leaflet </span>

*Version 2.0 March 10th, 2017*
#### &#9733;<i>Etienne David - Data Scientist :) &#9733;</i>
---

## From country to regional area ! - Applied to Sahel Area

Originally from Dr. Thomas Roca, AFD Data Officer 

The Demographic Health Surveys (DHS) are household surveys covering most of developping countries (and more) in the areas of population, health, and nutrition. These surveys are supported by <a href="https://www.usaid.gov/" target="_blank">USAID</a> and the <a href="https://www.census.gov/" target="_blank"> U.S. Census Bureau</a> (Kudos!)
More information about DHS: http://dhsprogram.com/

This script uses geolocated dataset to map DHS Data at the subnational level using <a href="http://leafletjs.com/" target="_blank">leaflet</a>.
More information about geotagged DHS data: http://spatialdata.dhsprogram.com/home/



![](http://spatialdata.dhsprogram.com/global/images/logo.png)

---

Packages used: fiona, json

### First we need to get the shapefile without indicator

When you download geoJSON from DHS, if the indicator doesn't exist at subnational level, their services won't provide you the shapefile... First we download shapefiles with... Employment characteristics (which is available at subnational level at least)

In [81]:
folder='sahel_2/shps/'
file_name='sdr_subnational_data'
nomdimension = 'Ceci est un test'
import fiona
import json 

#We use fiona package to convert the shapefile into json
with fiona.open(folder+file_name+'.shp') as source:
    records = list(source)
    geojson = {"type": "FeatureCollection","features": records}

with open(folder+file_name+".json", "w") as f:
    f.write(json.dumps(geojson))
    f = open(folder+file_name+".json",'r')

    filedata = f.read()

    f.close()
    
    
import json
filename=folder+file_name+'.json'
json_data=open(filename).read()
data = json.loads(json_data)

from string import Template
from IPython.display import HTML




vars()['json']=[]
    
    #loop over features lenght
for features in range (len(data["features"])-1):  
        content=data["features"][features]    
        vars()['json'].append(content)
        
        #write extra info used in leaflet
        string='{"type": "FeatureCollection", "features":'+ str(vars()['json']) + '}'
        #we replace None by string None
        string=string.replace('None', '"None"')
        #some information are coded using single quote, we replace the single quote by double
        string=string.replace("'",'"')
        #Save as json to be able to play with it in Python
        with open(folder+file_name+".json", "w") as f: f.write(string)
        
        #Save as js to map them with leaflet
        string='var statesData ='+string
        with open(folder+"sahel.js", "w") as f: f.write(string)

Meta data as presented on : http://spatialdata.dhsprogram.com/data/#/common/download


### Indicators:
- Households with electricity (i7C3CB1)
- Women with secondary or higher education (iA999FA)
- Women who are literate (i797B5DA)
- Total fertility rate (i133C8F8)
- Married women currently using any method of contraception (i209E190)
- Married women currently using any modern method of contraception (i209E191)
- Median age at first marriage: 25-49 (i349C430)
- Median age at first sexual intercourse: 25-49 (i36848B0)
- Unmet need total (iCE37FE2)
- Infant mortality rate (i42FFDB2)
- Under-five mortality rate (i42FFDB4)
- Place of delivery: Health facility (i49B3AD0)
- Received all vaccinations (i4E75F0A)
- Treatment of diarrhea: Either ORS or RHS (i533BDCB)
- Median duration of exclusive breastfeeding (i5716309)
- Children stunted (iCC96C1A)
- Children underweight (iCC97002)
- Children wasted (iCC973EA)
- Men receiving an HIV test and receiving test results in the last 12 months (i42759491)
- Women receiving an HIV test and receiving test results in the last 12 months (i42759492)
- HIV prevalence among men (i41075E91)
- HIV prevalence among women (i41075E92)
- HIV prevalence among general population (i41075E93)
- Children under 5 who slept under an insecticide treated net (ITN) (iC6D4E13)

## 2. List missing values

In [82]:
folder='sahel_2/shps/'
isolist=['AL','AO','AM','AZ','BD','BJ','BO','BT','BR','BF','BU','KH','CM','CF','TD','CO','KM','CG','CD','CI','DR','EC','EG','ES','ER','ET','GA','GH','GU','GN','GY','HT','HN','IA','ID','JO','KK','KE','KY','LS','LB','MD','MW','MV','ML','MR','MX','MB','MA','MZ','NM','NP','NC','NI','NG','PK','PY','PE','PH','RW','ST','SN','SL','ZA','LK','SD','SZ','TJ','TZ','TH','GM','TL','TG','TT','TN','TR','TM','UG','UA','UZ','VN','YE','ZM','ZW']
import json
# We use the indicators dictionnary that contains: the indicator code (e.g. i7C3CB1) the definition and 
# arbitrary min and max we set to define the range
 
ind_dict={"EMEMPLMDKM":"Percentage of men with 'don't know' or missing information on whether they worked in the last 12 months",\
"EMEMPLMEMC":"Percentage of men who worked in the 12 months preceding the survey and are working currently",\
"EMEMPLMENC":"Percentage of men who worked in the 12 months preceding the survey, but are not working currently",\
"EMEMPLMN12":"Percentage of men who did no work in the 12 months preceding the survey",\
"EMEMPLMNUM":"Number of men aged 15-49(54,59)",\
"EMEMPLMTOT":"Percentage of men: Total",\
"EMEMPLMUNW":"Number of men aged 15-49(54,59) (unweighted)",\
"EMEMPLWDKM":"Percentage of women with 'don't know' or missing information on whether they worked in the last 12 months",\
"EMEMPLWEMC":"Percentage of women who worked in the 12 months preceding the survey and are working currently",\
"EMEMPLWENC":"Percentage of women who worked in the 12 months preceding the survey, but are not working currently",\
"EMEMPLWN12":"Percentage of women who did no work in the 12 months preceding the survey",\
"EMEMPLWNUM":"Number of women aged 15-49",\
"EMEMPLWTOT":"Percentage of women: Total",\
"EMEMPLWUNW":"Number of women aged 15-49 (unweighted)",\
"EMOCCPMCLR":"Percentage of men employed in the 12 months before the survey whose occupation is clerical",\
"EMOCCPMPRO":"Percentage of men employed in the 12 months before the survey whose occupation is professional, technical, managerial",\
"EMOCCPMSAL":"Percentage of men employed in the 12 months before the survey whose occupation is sales, services",\
"EMOCCPWAGR":"Percentage of women employed in the 12 months before the survey whose occupation is in agriculture",\
"EMOCCPWCLR":"Percentage of women employed in the 12 months before the survey whose occupation is clerical",\
"EMOCCPWDKM":"Percentage of women employed in the 12 months before the survey with 'don't know' or missing information on occupation",\
"EMOCCPWDOM":"Percentage of women employed in the 12 months before the survey whose occupation is household & domestic",\
"EMOCCPWMNS":"Percentage of women employed in the 12 months before the survey whose occupation is skilled manual",\
"EMOCCPWMNU":"Percentage of women employed in the 12 months before the survey whose occupation is unskilled manual",\
"EMOCCPWNUM":"Number of women employed in the 12 months before the survey",\
"EMOCCPWOTH":"Percentage of women employed in the 12 months before the survey whose occupation is other",\
"EMOCCPWPRO":"Percentage of women employed in the 12 months before the survey whose occupation is professional, technical, managerial",\
"EMOCCPWSAL":"Percentage of women employed in the 12 months before the survey whose occupation is sales, services",\
"EMOCCPWTOT":"Percentage of women employed in the 12 months before the survey: Total",\
"EMOCCPWUNW":"Number of women employed in the 12 months before the survey (unweighted)"}
Indicator_list= list(ind_dict.keys())

#create en empty list to host the missing values list
string=[]

# Loop over countries
#open json files
json_data=open(folder+file_name+'.json').read()
print(folder+file_name)
data0 = json.loads(json_data)

dict_min ={key: None for key in Indicator_list}
dict_max={key: None for key in Indicator_list}
pays = []
print(dict_min)

    
#loop over dictionnary values
for code,info_ind in ind_dict.items():
    item=info_ind.split(',')
    definition=item[0]
        
    #Read the features
    for features in range (0,len(data0["features"])-1):
        Region=data0["features"][features]["properties"]["DHSREGEN"]
        nb = data0["features"][features]["properties"][code]
        iso = data0["features"][features]["properties"]["ISO"]
    
        
  
        if nb ==9999 :
            #Append the list of indicators
            string.append(iso+','+Region+','+code+','+definition+',no data\n')
        else:
            if dict_max[code] is None:
                dict_max[code] = nb
            elif dict_min[code] is None:
                dict_min[code] = nb
            elif(nb > dict_max[code]):
                dict_max[code] = nb
            elif (nb < dict_min[code]):
                dict_min[code] = nb
                
        if iso not in pays:
            pays.append(iso)
                
            
      
            
            
                
# Finally Write the missing values list into a csv file 
with open(folder+'missing.csv', "w") as f: f.write(''.join(map(str,string)))
print(dict_max)
print(dict_min)
print(pays)

sahel_2/shps/sdr_subnational_data
{'EMEMPLMDKM': None, 'EMEMPLMEMC': None, 'EMEMPLMENC': None, 'EMEMPLMN12': None, 'EMEMPLMNUM': None, 'EMEMPLMTOT': None, 'EMEMPLMUNW': None, 'EMEMPLWDKM': None, 'EMEMPLWEMC': None, 'EMEMPLWENC': None, 'EMEMPLWN12': None, 'EMEMPLWNUM': None, 'EMEMPLWTOT': None, 'EMEMPLWUNW': None, 'EMOCCPMCLR': None, 'EMOCCPMPRO': None, 'EMOCCPMSAL': None, 'EMOCCPWAGR': None, 'EMOCCPWCLR': None, 'EMOCCPWDKM': None, 'EMOCCPWDOM': None, 'EMOCCPWMNS': None, 'EMOCCPWMNU': None, 'EMOCCPWNUM': None, 'EMOCCPWOTH': None, 'EMOCCPWPRO': None, 'EMOCCPWSAL': None, 'EMOCCPWTOT': None, 'EMOCCPWUNW': None}
{'EMEMPLMDKM': 2.8, 'EMEMPLMEMC': 99.6, 'EMEMPLMENC': 38.0, 'EMEMPLMN12': 42.9, 'EMEMPLMNUM': 1116.0, 'EMEMPLMTOT': 100.0, 'EMEMPLMUNW': 738.0, 'EMEMPLWDKM': 0.4, 'EMEMPLWEMC': 93.4, 'EMEMPLWENC': 12.1, 'EMEMPLWN12': 95.0, 'EMEMPLWNUM': 2561.0, 'EMEMPLWTOT': 100.0, 'EMEMPLWUNW': 2590.0, 'EMOCCPMCLR': 9.8, 'EMOCCPMPRO': 13.8, 'EMOCCPMSAL': 41.2, 'EMOCCPWAGR': 80.5, 'EMOCCPWCLR': 8.2,

# Create Maps

In [83]:
import json
from string import Template
from IPython.display import HTML
import numpy as np


# We created a GPS dictionnary to store: the gps location of the capital city to center the map so as the zoom level
gps_dict={"AL":"ALB,41.1533,20.1683,7", "AO":"AGO,-11.2027,17.8739,5", "AM":"ARM,40.0691,45.0382,7", "AZ":"AZE,40.1431,47.5769,7",\
          "BD":"BGD,23.685,90.3563,7", "BJ":"BEN,9.30769,2.31583,6", "BO":"BOL,-16.2902,-63.5887,5", "BT":"BTN,27.5142,90.4336,8",\
          "BR":"BRA,-14.235,-51.9253,4", "BF":"BFA,12.2383,-1.56159,6", "BU":"null,-32.87,26.12,6", "KH":"KHM,12.5657,104.991, 7",\
          "CM":"CMR,7.36972,12.3547, 5", "CF":"CAF,6.61111,20.9394,6", "TD":"TCD,15.4542,18.7322,5", "CO":"COL,4.57087,-74.2973,5",\
          "KM":"COM,-11.875,43.8722,8", "CG":"COG,-0.228021,15.8277,6", "CD":"COD,-4.03833,21.7587,5", "CI":"CIV,7.53999,-5.54708,6",\
          "DR":"null,-32.87,26.12,7", "EC":"ECU,-1.83124,-78.1834,6", "EG":"EGY,26.8206,30.8025,5", "ES":"ESP,40.4637,-3.74922,6",\
          "ER":"ERI,15.1794,39.7823,7", "ET":"ETH,9.145,40.4897,5", "GA":"GAB,-0.803689,11.6094,6", "GH":"GHA,7.94653,-1.02319,6",\
          "GU":"GUM,13.4443,144.794,9", "GN":"GIN,9.94559,-9.69664,6", "GY":"GUY,4.86042,-58.9302,6", "HT":"HTI,18.9712,-72.2852,8",\
          "HN":"HND,15.2,-86.2419,7", "IA":"null,-32.87,26.12,7", "ID":"IDN,-0.789275,113.921,4", "JO":"JOR,30.5852,36.2384,7",\
          "KK":"null,-32.87,26.12,7", "KE":"KEN,-0.023559,37.9062,6", "KY":"CYM,19.5135,-80.567,9", "LS":"LSO,-29.61,28.2336,8",\
          "LB":"LBN,33.8547,35.8623,8", "MD":"MDA,47.4116,28.3699,7", "MW":"MWI,-13.2543,34.3015,6", "MV":"MDV,3.20278,73.2207,7",\
          "ML":"MLI,17.5707,-3.99617,5", "MR":"MRT,21.0079,-10.9408,5", "MX":"MEX,23.6345,-102.553,5", "MB":"null,-32.87,26.12,7",\
          "MA":"MAR,31.7917,-7.09262,5", "MZ":"MOZ,-18.6657,35.5296,5", "NE": "NER, 13.51, 2.12, 5", "NM":"null,-32.87,26.12,7", "NP":"NPL,28.3949,84.124,7",\
          "NC":"NCL,-20.9043,165.618,7", "NI":"NIC,12.8654,-85.2072,7", "NG":"NGA,9.082,8.67528,6", "PK":"PAK,30.3753,69.3451,5",\
          "PY":"PRY,-23.4425,-58.4438,6", "PE":"PER,-9.18997,-75.0152,5", "PH":"PHL,12.8797,121.774,5", "RW":"RWA,-1.94028,29.8739,8",\
          "ST":"STP,0.18636,6.61308,8", "SN":"SEN,14.4974,-14.4524,7", "SL":"SLE,8.46056,-11.7799,7", "ZA":"ZAF,-30.5595,22.9375,5",\
          "LK":"LKA,7.87305,80.7718,7", "SD":"SDN,12.8628,30.2176,5", "SZ":"SWZ,-26.5225,31.4659,8", "TJ":"TJK,38.861,71.2761,6",\
          "TZ":"TZA,-6.36903,34.8888,5", "TH":"THA,15.87,100.993,5", "GM":"GMB,13.4432,-15.3101,8", "TL":"TLS,-8.87422,125.728,8",\
          "TG":"TGO,8.61954,0.824782,7", "TT":"TTO,10.6918,-61.2225,8", "TN":"TUN,33.8869,9.5375,6", "TR":"TUR,38.9637,35.2433,6",\
          "TM":"TKM,38.9697,59.5563,7", "UG":"UGA,1.37333,32.2903,7", "UA":"UKR,48.3794,31.1656,6", "UZ":"UZB,41.3775,64.5853,6",\
          "VN":"VNM,14.0583,108.277,6", "YE":"YEM,15.5527,48.5164,6", "ZM":"ZMB,-13.1339,27.8493,6", "ZW":"ZWE,-19.0154,29.1549,6"}

#We get the center of countries

bary =np.array([0,0])
zoomlevel = 10
for i in pays:
    t = gps_dict[i]
    
    item = t.split(',')
    print(float(item[1]))
    bary[0] += float(item[1])
    bary[1] += float(item[2])
    zoomlevel = min(zoomlevel, float(item[3]))

bary = bary /len(pays)

Longitude=item[1]
Latitude=item[2]

print(zoomlevel)



#=====================================================================================================================
#                                         ! Done
#=====================================================================================================================

12.2383
17.5707
21.0079
13.51
5.0


In [84]:
#OKKKKK Now we have to define one style per indicator ! Let's Do it ! 
text = ''
baseMaps =''
dict_Maps =''
mapon =''
variables = ''
from string import Template


Indicator_list= list(ind_dict.keys())



for indicator in Indicator_list:
    # save indicator definition
    definition=ind_dict[indicator]

    baseMaps += '"'+definition+'":'+indicator+","
    dict_Maps += '"'+definition+'":"'+indicator+'",'
    min_ind= dict_min[indicator]
    max_ind= dict_max[indicator]
    #Set color range
    ind_scale=(max_ind-min_ind)/8
    cat0=0
    for val in range(1,7+1):
        vars()['cat'+str(val)]=round(ind_scale+vars()['cat'+str(val-1)])

    Input = {'definition': definition.replace("'",""), 'indicator': indicator, 'cat1':cat1, 'cat2':cat2, 'cat3':cat3,'cat4':cat4,'cat5':cat5,'cat6':cat6, 'cat7':cat7 }   
    if not variables:
        variables = Template('var $indicator = L.geoJson(statesData, {style: style$indicator, onEachFeature: onEachFeature$indicator}).addTo(map);').safe_substitute(Input)
    else:
        
        variables = variables + Template('''
            var $indicator = L.geoJson(statesData, {style: style$indicator, onEachFeature: onEachFeature$indicator});
        ''').safe_substitute(Input)
    texto = '''
                    function resetHighlight$indicator(e) {
            
                $indicator.resetStyle(e.target);
                info.update();
            }
            
            function onEachFeature$indicator(feature, layer) {
                layer.on({
                    mouseover: highlightFeature,
                    mouseout: resetHighlight$indicator,
                    click: zoomToFeature
                });
            }
                function getColor$indicator(d) {
                            return d==9999 ? '#EEEEEE' :
                                   d > $cat7 ? '#1a334c' :
                                   d > $cat6  ? '#084594' :
                                   d > $cat5  ? '#2171b5' :
                                   d > $cat4  ? '#4292c6' :
                                   d > $cat3   ? '#6baed6' :
                                   d > $cat2   ? '#9ecae1' :
                                   d > $cat1   ? '#c6dbef' :
                                              '#eff3ff';
                        }

                        function style$indicator(feature) {
                            return {
                                weight: 2,
                                opacity: 1,
                                color: 'white',
                                dashArray: '3',
                                fillOpacity: 0.7,
                                fillColor: getColor$indicator(feature.properties.$indicator)
                            };
                        }
                        
                        
                 var legend$indicator = L.control({position: 'bottomright'});

                        legend$indicator.onAdd = function (map) {

                            var div = L.DomUtil.create('div', 'info legend'),
                                grades = [$cat1,$cat2,$cat3,$cat4,$cat5,$cat6,$cat7],
                                labels = [],
                                from, to;

                            for (var i = 0; i < grades.length; i++) {
                                from = grades[i];
                                to = grades[i + 1];

                                labels.push(
                                    '<i style="background:' + getColor$indicator(from + 1) + '"></i> ' +
                                    from + (to ? '&ndash;' + to : '+'));
                            }

                            div.innerHTML = labels.join('<br>');
                            return div;
                        };
                        
                        '''
    
    file_content = Template(texto).safe_substitute(Input)
        
    text +='\n'+file_content
    
    if not mapon:
        temp = '''
        map.on('baselayerchange', function (eventLayer) {
        doc = eventLayer.name;
        indicator = dict_Maps[eventLayer.name];
        if (eventLayer.name === '$definition') {
        map.removeControl(currentLegend );
        currentLegend = legend$indicator;
        legend$indicator.addTo(map);
        }
        '''
        mapon = Template(temp).safe_substitute(Input)
    else:
        mapon += Template(''' 
            else if  (eventLayer.name === '$definition') {
        map.removeControl(currentLegend );
        currentLegend = legend$indicator;
        legend$indicator.addTo(map);
    }''').safe_substitute(Input)

#.addTo(map);

mapon= mapon+'})'
baseMaps = '{' + baseMaps[:-1] + '}'
dict_Maps = '{' + dict_Maps[:-1] + '}'



In [85]:

Input = {'Namedimension': nomdimension,'filenamejs': "sahel.js", 'Longitude': Longitude, 'Latitude':Latitude, 'zoomlevel':zoomlevel, 'indicator':Indicator_list[0], 'definition': ind_dict[Indicator_list[0]].replace("'","")}   




header = '''
                  <!DOCTYPE html>
            <html><head>
                    <title>Leaflet Layers Control Example</title>
                    <meta charset="utf-8" />
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <link rel="stylesheet" href="https://npmcdn.com/leaflet@1.0.0-rc.3/dist/leaflet.css" />
                    <style>
                        #map {
                            width: 1080px;
                            height: 960px;
                        }

                        .info {
                            padding: 6px 8px;
                            font: 14px/16px Arial, Helvetica, sans-serif;
                            background: white;
                            background: rgba(255,255,255,0.8);
                            box-shadow: 0 0 15px rgba(0,0,0,0.2);
                            border-radius: 5px;
                        }
                        .info h4 {
                            margin: 0 0 5px;
                            color: #777;
                        }

                        .legend {
                            text-align: left;
                            line-height: 18px;
                            color: #555;
                        }
                        .legend i {
                            width: 18px;
                            height: 18px;
                            float: left;
                            margin-right: 8px;
                            opacity: 0.7;
                        }
                    </style>
                </head>
                
                <body>
                    <div id="map"></div>

                    <script src="https://npmcdn.com/leaflet@1.0.0-rc.3/dist/leaflet.js"></script>

                    <script type="text/javascript" src="$filenamejs"></script>
                    <script type="text/javascript">

                        var map = L.map('map').setView([$Longitude,$Latitude], $zoomlevel);

                        L.tileLayer('https://api.tiles.mapbox.com/v4/{id}/{z}/{x}/{y}.png?access_token=pk.eyJ1IjoibWFwYm94IiwiYSI6ImNpejY4NXVycTA2emYycXBndHRqcmZ3N3gifQ.rJcFIG214AriISLbB6B5aw',
                            {maxZoom: 18,
                            attribution: 'Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors, ' +
                                '<a href="http://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, ' +
                                'Dataviz from <a href="https://twitter.com/Thomas_Roca">@Thomas_Roca </a> with Python',
                            id: 'mapbox.light'
                        }).addTo(map);
                        
                                    var info = L.control();

            info.onAdd = function (map) {
                this._div = L.DomUtil.create('div', 'info');
                this.update();
                return this._div;
            };

            info.update = function (props) {
                if(props){
                if (props[indicator] === 9999) {
                var t = 'Missing Value';
                } else {
                var t = props[indicator];
                }}

                this._div.innerHTML = (props ?
               '<h3>'+props.CNTRYNAMEE+'</h3>' +
                '<span style="color:#336DA9"><b>'+doc+': '+
                '<span style="color:black">' + t +'</b></span> %<br><span style="font-size:12px;color:black ">'+
                'Survey Region: <b>' + props.DHSREGEN + '</b><br>' +
                'Survey Year: <b> '+props.SVYYEAR + '</b><br>' +
                '</b> <i>Source: <b>DHS program</i></b></a>'
                : '<b>$Namedimension</b>');
            };
        

            info.addTo(map);
                    

                       map.attributionControl.addAttribution('Data &copy; <a href="http://spatialdata.dhsprogram.com/" target="_blank">DHS Spatial Data Repository</a>');
                       
                '''
highlight = '''

            function highlightFeature(e) {
                var layer = e.target;

                layer.setStyle({
                    weight: 5,
                    color: '#666',
                    dashArray: '',
                    fillOpacity: 0.7
                });

                if (!L.Browser.ie && !L.Browser.opera) {
                    layer.bringToFront();
                }

                info.update(layer.feature.properties);
            }




            function zoomToFeature(e) {
                map.fitBounds(e.target.getBounds());
            }

       
'''
overlay = '''
var overlayMaps ={}
    
    L.control.layers(baseMaps, overlayMaps, {
    collapsed:true,
    position:'bottomleft'
}).addTo(map);
legend$indicator.addTo(map);
currentLegend = legend$indicator;
var doc = '$definition';
var indicator = '$indicator';




''' 

header = Template(header).safe_substitute(Input)
highlight = Template(highlight).safe_substitute(Input)

overlay = Template(overlay).safe_substitute(Input)

html = header + variables + text + highlight + 'var baseMaps = '+baseMaps+'; var dict_Maps = '+dict_Maps+';'+ overlay + mapon +     '''
                </script>
            </body>
            </html> '''

In [86]:
#open an handle
f = open(folder+"sahel.html",'w')
#write the content within the handle
file_content = Template(html).safe_substitute(Input)
f.write(file_content)
f.close()